# specific example analysis

Investigate few samples with short API sequences (obviously, anti-debugging / anti-emulation techniques ):

In [1]:
import speakeasy
import pandas as pd
import json

MALWARE_PATH = "../../../data/pe.dataset/"
X86_PATH = MALWARE_PATH + "PeX86Exe/"
X86_RANSOMWARE = X86_PATH + "ransomware/"
X86_CLEAN = X86_PATH + "clean/"
X86_DROPPER = X86_PATH + "dropper/"


def get_apis_n_errors(fullpath):
    se = speakeasy.Speakeasy(config=json.load(open("./config.json")))
    module = se.load_module(fullpath)
    se.run_module(module)

    report = se.get_report()
    aaa = pd.json_normalize(report["entry_points"])
    errors = aaa[[x for x in aaa.columns if "error" in x]]
    return pd.json_normalize([x["apis"] for x in report["entry_points"]][0]), errors


In [2]:
#a, e = get_apis_n_errors("../../../data/pe.dataset/PeX86Exe/dropper/61a53907ed506b8881ede7a47a9d48a1b650b762af131c94250824036db8ad64")
a, e = get_apis_n_errors("../../../data/pe.dataset/PeX86Exe/dropper/eb70ef39f101faafde59dd1551ac1bd86d2cc8c586ec3acb144f7a928e2d1aa0")
a.iloc[-10:]

,pc,api_name,args,ret_val
789,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x45f0, 0x4610]",0x0
790,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x0, 0x1]",0x0
791,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x2, 0x1211fd0]",0x0
792,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x0, 0x0]",0x0
793,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x1211f6c, 0xd27d571f]",0x0
794,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x0, 0x405d90]",0x0
795,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x1fa15775, 0x1]",0x0
796,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x1212000, 0xfeedf000]",0x0
797,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x4000, 0x4010]",0x0
798,0x402296,KERNEL32.GetConsoleTitleW,"[0x0, 0x0, 0x4020, 0x4030]",0x0


In [5]:
print(e["error.traceback"].iloc[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/windows/winemu.py", line 437, in start
    count=self.max_instructions)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/engines/unicorn_eng.py", line 203, in start
    return self.emu.emu_start(addr, 0xFFFFFFFF, timeout=timeout, count=count)
  File "/usr/local/lib/python3.6/site-packages/unicorn/unicorn.py", line 318, in emu_start
    raise UcError(status)
unicorn.unicorn.UcError: Invalid memory fetch (UC_ERR_FETCH_UNMAPPED)



In [2]:
a, e = get_apis_n_errors(X86_CLEAN + "1cb157e6db12dc2871720ad4104d03f09b0cb3dfbc5e28d31bc2ce66cf760409")
a

,pc,api_name,args,ret_val
0,0x401795,MSVCRT.__set_app_type,[0x2],None
1,0x4017b2,MSVCRT.__p__fmode,[],0x4610
2,0x4017c0,MSVCRT.__p__commode,[],0x4620
3,0x40192f,MSVCRT._controlfp,"[0x10000, 0x30000]",0x0
4,0x401805,MSVCRT._initterm,"[0x403008, 0x40300c]",0x0
5,0x40182e,MSVCRT.__getmainargs,"[0x1211f84, 0x1211f74, 0x1211f80, 0x0, 0x1211f78]",0x0
6,0x401840,MSVCRT._initterm,"[0x403000, 0x403004]",0x0
7,0x40188f,KERNEL32.GetStartupInfoA,[0x1211f88],None
8,0x4018b0,KERNEL32.GetModuleHandleA,[0x0],0x400000
9,0x4014a3,KERNEL32.GetModuleFileNameA,"[0x400000, C:\Windows\system32\1cb157e6db12dc2...",0x54


In [3]:
e

,error.type,error.pc,error.address,error.instr,error.regs.esp,error.regs.ebp,error.regs.eip,error.regs.esi,error.regs.edi,error.regs.eax,error.regs.ebx,error.regs.ecx,error.regs.edx,error.stack,error.traceback
0,Invalid memory read (UC_ERR_READ_UNMAPPED),0x401960,0x401960,jmp dword ptr [0x40209c],0x012118f4,0x01211a10,0x00401960,0x004031c0,0x00000001,0x00e0e735,0xffffffff,0x00000000,0x00000001,"[sp+0x00: 0xfeedf004, sp+0x04: 0x000046c0 -> e...","Traceback (most recent call last):\n File ""/u..."


In [4]:
print(e["error.traceback"].iloc[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/windows/winemu.py", line 437, in start
    count=self.max_instructions)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/engines/unicorn_eng.py", line 203, in start
    return self.emu.emu_start(addr, 0xFFFFFFFF, timeout=timeout, count=count)
  File "/usr/local/lib/python3.6/site-packages/unicorn/unicorn.py", line 318, in emu_start
    raise UcError(status)
unicorn.unicorn.UcError: Invalid memory read (UC_ERR_READ_UNMAPPED)



In [2]:
a,e = get_apis_n_errors(X86_DROPPER + "6c68b4eae540b739e9a206e997986c88e6d4c602b6603550365aa4a331f65601")
a

,pc,api_name,args,ret_val
0,0x401267,CRTDLL.__GetMainArgs,"[0x406020, 0x406024, 0x406028, 0x0, 0x0]",0x0
1,0x4015af,iphlpapi.GetAdaptersInfo,"[0x1211a88, 0x1211e14, 0x0, 0x0, 0x0]",0x0
2,0x4016bd,KERNEL32.GetLastError,[],0x0
3,0x4016c4,USER32.GetCursorInfo,[0x0],0x0


In [3]:
e

,error.type,error.pc,error.address,error.instr,error.regs.esp,error.regs.ebp,error.regs.eip,error.regs.esi,error.regs.edi,error.regs.eax,error.regs.ebx,error.regs.ecx,error.regs.edx,error.stack,error.traceback
0,Invalid memory read (UC_ERR_READ_UNMAPPED),0x40109a,0x40109a,cld,0x01211d54,0x01211e10,0x0040109a,0x00068db8,0x00000094,0x01211d84,0xffffffff,0x00002710,0x00068db7,"[sp+0x00: 0xfeedf004, sp+0x04: 0x00004600 -> e...","Traceback (most recent call last):\n File ""/u..."


In [4]:
print(e["error.traceback"].iloc[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/windows/winemu.py", line 437, in start
    count=self.max_instructions)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/engines/unicorn_eng.py", line 203, in start
    return self.emu.emu_start(addr, 0xFFFFFFFF, timeout=timeout, count=count)
  File "/usr/local/lib/python3.6/site-packages/unicorn/unicorn.py", line 318, in emu_start
    raise UcError(status)
unicorn.unicorn.UcError: Invalid memory read (UC_ERR_READ_UNMAPPED)



In [2]:
a, e = get_apis_n_errors(X86_DROPPER + "88909c6e4f84ed4c6e30cbb9cee8c236b44540524e504f33477120d1af7352f6")
a

<class 'int'>


,pc,api_name,args,ret_val
0,0x4010f6,MSVBVM60.ordinal_100,[0x4012ac],0x4012ac


In [3]:
e

,error.type,error.pc,error.address,error.instr,error.regs.esp,error.regs.ebp,error.regs.eip,error.regs.esi,error.regs.edi,error.regs.eax,error.regs.ebx,error.regs.ecx,error.regs.edx,error.stack
0,invalid_read,0x401107,0x1004af3,"add byte ptr [ecx + eax*4 + 0x3f], ch",0x01211fe8,0x01212000,0x00401107,0x00000000,0x00000000,0x004012ad,0x00000000,0x00000000,0x00000000,"[sp+0x00: 0xfeedf000, sp+0x04: 0x00004000 -> e..."


In [2]:
a, e = get_apis_n_errors(X86_CLEAN + "d5b71fa5970386eda8426498171cfd8b3623e1a19f4a918491115a7c88811a64")
a

,pc,api_name,args,ret_val
0,0x401795,MSVCRT.__set_app_type,[0x2],None
1,0x4017b2,MSVCRT.__p__fmode,[],0x4610
2,0x4017c0,MSVCRT.__p__commode,[],0x4620
3,0x40192f,MSVCRT._controlfp,"[0x10000, 0x30000]",0x0
4,0x401805,MSVCRT._initterm,"[0x403008, 0x40300c]",0x0
5,0x40182e,MSVCRT.__getmainargs,"[0x1211f84, 0x1211f74, 0x1211f80, 0x0, 0x1211f78]",0x0
6,0x401840,MSVCRT._initterm,"[0x403000, 0x403004]",0x0
7,0x40188f,KERNEL32.GetStartupInfoA,[0x1211f88],None
8,0x4018b0,KERNEL32.GetModuleHandleA,[0x0],0x400000
9,0x4014a3,KERNEL32.GetModuleFileNameA,"[0x400000, C:\Windows\system32\d5b71fa5970386e...",0x54


In [4]:
e

,error.type,error.pc,error.address,error.instr,error.regs.esp,error.regs.ebp,error.regs.eip,error.regs.esi,error.regs.edi,error.regs.eax,error.regs.ebx,error.regs.ecx,error.regs.edx,error.stack,error.api_name
0,unsupported_api,0xfeedf06c,0xfeedf06c,disasm_failed,0x01211b00,0x01211b34,0xfeedf06c,0x00000080,0x004031c0,0x00000080,0xfeedf06c,0x00000080,0x00004041,[sp+0x00: 0x004016cb -> emu.module.d5b71fa5970...,KERNEL32._llseek


In [2]:
a,e = get_apis_n_errors(X86_RANSOMWARE + "d042606747e507e80bdee11f13d40c167a8ad2d8adfdef9114f0cfb1965aa2e0")
a

,pc,api_name,args,ret_val
0,0x401005,kernel32.GetConsoleWindow,[],0x198
1,0x40101b,user32.GetCaretBlinkTime,[],0x3e8
2,0x401031,user32.GetFocus,[],0x1a0
3,0x5188a3,kernel32.GetSystemDefaultUILanguage,[],0x409
4,0x5188cb,kernel32.GetSystemDefaultUILanguage,[],0x409
5,0x518734,kernel32.GetUserDefaultLCID,[],0x400
6,0x5183af,kernel32.GetUserDefaultLCID,[],0x400
7,0x5189af,kernel32.GetUserDefaultLCID,[],0x400
8,0x518907,kernel32.GetUserDefaultLCID,[],0x400
9,0x518268,kernel32.GetUserDefaultLCID,[],0x400


In [3]:
e

""
0


In [2]:
a,e = get_apis_n_errors(X86_RANSOMWARE + "37f5efd1aed9594306f7d26841829d6f11d0e73efc68eeda0ebde9ee1f1cfe75")
a

,pc,api_name,args,ret_val
0,0x401008,kernel32.GetProcessHeap,[],0x45f0
1,0x401013,user32.GetMessageTime,[],0x602a6137


In [3]:
e

""
0


In [14]:
a, e = get_apis_n_errors("../../../data/pe.dataset/PeX86Exe/ransomware/8bf742a7ff63433089f69de0cca6007257545d068a1e7d5ab92ebe2302825741")

In [15]:
a

,pc,api_name,args,ret_val
0,0x401008,kernel32.GetCurrentThreadId,[],0xb00
1,0x401013,kernel32.GetCurrentThreadId,[],0xb00


In [16]:
e

""
0


In [2]:
a, e = get_apis_n_errors("../../../data/pe.dataset/PeX86Exe/ransomware/a2d6949a5a0aeef96eeb2c18e4ac7f642b72f0741a0c86518c816a29dae98430")
a

,pc,api_name,args,ret_val
0,0x401008,user32.GetFocus,[],0x1a0
1,0x401013,user32.GetFocus,[],0x1a4


In [3]:
e

""
0


In [4]:
a, e = get_apis_n_errors("../../../data/pe.dataset/PeX86Exe/ransomware/e76b579c0f7c849d3dcf910366e28e7943214f1f1fe9c26e51446520a959d841")

In [5]:
a

,pc,api_name,args,ret_val
0,0x401008,kernel32.GetCurrentThreadId,[],0x528
1,0x401013,user32.GetMessagePos,[],0x19999999


In [6]:
e

""
0


In [2]:
a,e = get_apis_n_errors("../../../data/pe.dataset/PeX86Exe/ransomware/9c52bb6f877d0439b6bd428ebac249588f8bda566fc6b3c5fb5ebd8cd0cfcaba")
a

,pc,api_name,args,ret_val
0,0x401008,user32.GetCursor,[],0x1a0
1,0x401013,user32.GetClipboardSequenceNumber,[],0x0


In [3]:
e

""
0
